In [1]:
import logging
import os
import pickle
import time
import statistics
from abc import abstractmethod

import colorsys
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import hdbscan

from sklearn.cluster import OPTICS, cluster_optics_dbscan
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from sklearn.metrics import silhouette_score, silhouette_samples
import statistics

from sklearn.neighbors import LocalOutlierFactor

from sklearn.svm import OneClassSVM




In [2]:
f0 = 'base.csv'
f1 = 'random1.csv'
f2 = 'wind1.csv'
f3 = 'engine1.csv'
f4 = 'sensor1.csv'
f5 = 'crash.csv'

base_df = pd.read_csv(f0, header=0, index_col=None)
random1_df = pd.read_csv(f1, header=0, index_col=None)
wind1_df = pd.read_csv(f2, header=0, index_col=None)
engine1_df = pd.read_csv(f3, header=0, index_col=None)
sensor1_df = pd.read_csv(f4, header=0, index_col=None)
crash1_df = pd.read_csv(f5, header=0, index_col=None)

df_list = [base_df, random1_df, wind1_df, engine1_df, sensor1_df,crash1_df]

for df in df_list:

    df['airspeedchange'] = df['airspeed']- df['airspeed'].shift(1)
    df['airspeedchange'].fillna(0, inplace=True)

base_df = base_df[['roll', 'pitch', 'yaw', 'rollspeed', 'pitchspeed', 'yawspeed',  'airspeedchange']]
clustering_random1_df = random1_df[['roll', 'pitch', 'yaw', 'rollspeed', 'pitchspeed', 'yawspeed', 'airspeedchange']]
clustering_wind1_df = wind1_df[['roll', 'pitch', 'yaw', 'rollspeed', 'pitchspeed', 'yawspeed', 'airspeedchange']]
clustering_sensor1_df = sensor1_df[['roll', 'pitch', 'yaw', 'rollspeed', 'pitchspeed', 'yawspeed', 'airspeedchange']]
clustering_crash1_df = crash1_df[['roll', 'pitch', 'yaw', 'rollspeed', 'pitchspeed', 'yawspeed', 'airspeedchange']]
clustering_engine1_df = engine1_df[['roll', 'pitch', 'yaw', 'rollspeed', 'pitchspeed', 'yawspeed', 'airspeedchange']]



In [3]:
#train the 5 models

kmeans = KMeans(init="random", n_clusters=2, n_init=10, random_state=1)
kmeans.fit(base_df)

lof = LocalOutlierFactor(n_neighbors=14, contamination="auto",novelty=True)
lof.fit(base_df)

nu = 0.1
kernel = 'rbf'
gamma = 'scale'

model = OneClassSVM(nu=nu, kernel=kernel, gamma=gamma)
model.fit(base_df)

db = DBSCAN(eps = 0.78).fit(base_df)

optics_clustering = OPTICS(min_samples = 50).fit(base_df)



In [4]:
def initialisation_rule_checker(datapoint):
    rules_broken = 0
    if datapoint["climb"] > 0.04 or datapoint["climb"] < -0.02:
        rules_broken += 1
    
    if datapoint["airspeed"]!= 0.0:
        rules_broken += 1

    if datapoint["throttle"]!= 0.0:
        rules_broken += 1

    if datapoint["roll"] > -0.0003 or datapoint["roll"] < -0.0009:
        rules_broken += 1

    if datapoint["pitch"] > 0.001 or datapoint["pitch"] < 0.0009:
        rules_broken += 1

    if datapoint["pitchspeed"] > 0.0005 or datapoint["pitchspeed"] < 0.0001:
        rules_broken += 1

    if datapoint["rollspeed"] > 0.0004 or datapoint["rollspeed"] < 0.00009:
        rules_broken += 1

    if datapoint["yawspeed"] > 0.002 or datapoint["yawspeed"] < 0.001:
        rules_broken += 1
    return rules_broken

def takeoff_rule_checker(datapoint):
    rules_broken = 0
    if datapoint["climb"] > 1.6 or datapoint["climb"] < -0.2:
        rules_broken += 1
    
    if datapoint["airspeed"]>6 or datapoint["airspeed"] < 0:
        rules_broken += 1

    if datapoint["throttle"]> 80 or datapoint["throttle"] <0:
        rules_broken += 1

    if datapoint["roll"] > 0.17 or datapoint["roll"] < -0.07:
        rules_broken += 1

    if datapoint["pitch"] > 0.16 or datapoint["pitch"] < -0.19:
        rules_broken += 1

    if datapoint["pitchspeed"] > 0.1 or datapoint["pitchspeed"] < -0.15:
        rules_broken += 1

    if datapoint["rollspeed"] > 0.89 or datapoint["rollspeed"] < -0.87:
        rules_broken += 1

    if datapoint["yawspeed"] > 0.85 or datapoint["yawspeed"] < -0.095:
        rules_broken += 1
    return rules_broken

def on_mission_rule_checker(datapoint):
    rules_broken = 0
    if datapoint["climb"] > 1.6 or datapoint["climb"] < -1.1:
        rules_broken += 1
    
    if datapoint["airspeed"]>7.05 or datapoint["airspeed"] < 0.1:
        rules_broken += 1

    if datapoint["throttle"]> 100 or datapoint["throttle"] <31:
        rules_broken += 1

    if datapoint["roll"] > 0.22 or datapoint["roll"] < -0.25:
        rules_broken += 1

    if datapoint["pitch"] > 0.16 or datapoint["pitch"] < -0.3:
        rules_broken += 1

    if datapoint["pitchspeed"] > 0.2 or datapoint["pitchspeed"] < -0.22:
        rules_broken += 1

    if datapoint["rollspeed"] > 1.07 or datapoint["rollspeed"] < -1.14:
        rules_broken += 1

    if datapoint["yawspeed"] > 0.79 or datapoint["yawspeed"] < -0.81:
        rules_broken += 1
    return rules_broken

def return_to_origin_rule_checker(datapoint):
    rules_broken = 0
    if datapoint["climb"] > 0.09  or datapoint["climb"] < -0.041:
        rules_broken += 1
    
    if datapoint["airspeed"]> 6.9 or datapoint["airspeed"] < 0.04:
        rules_broken += 1

    if datapoint["throttle"]> 50 or datapoint["throttle"] < 32:
        rules_broken += 1

    if datapoint["roll"] > 0.068 or datapoint["roll"] < -0.07:
        rules_broken += 1

    if datapoint["pitch"] > 0.22 or datapoint["pitch"] < -0.062:
        rules_broken += 1

    if datapoint["pitchspeed"] > 0.088 or datapoint["pitchspeed"] < -0.087:
        rules_broken += 1

    if datapoint["rollspeed"] > 0.68 or datapoint["rollspeed"] < -0.68:
        rules_broken += 1

    if datapoint["yawspeed"] > 0.039 or datapoint["yawspeed"] < -0.043:
        rules_broken += 1
    return rules_broken

def landing_rule_checker(datapoint):
    rules_broken = 0
    if datapoint["climb"] > 0.163 or datapoint["climb"] < -1.18:
        rules_broken += 1
    
    if datapoint["airspeed"]>6.1 or datapoint["airspeed"] < 0:
        rules_broken += 1

    if datapoint["throttle"]> 38 or datapoint["throttle"] <0:
        rules_broken += 1

    if datapoint["roll"] > 0.05 or datapoint["roll"] < -0.081:
        rules_broken += 1

    if datapoint["pitch"] > 0.05 or datapoint["pitch"] < -0.07:
        rules_broken += 1

    if datapoint["pitchspeed"] > 0.04 or datapoint["pitchspeed"] < -0.05:
        rules_broken += 1

    if datapoint["rollspeed"] > 1.61 or datapoint["rollspeed"] < -1.49:
        rules_broken += 1

    if datapoint["yawspeed"] > 0.071 or datapoint["yawspeed"] < -0.023:
        rules_broken += 1
    return rules_broken

def constant_rule_checker(datapoint):
    rules_broken = 0
    if datapoint["GPS_status"] != 1:
        rules_broken += 1

    if datapoint["Gyro_status"] != 1:
        rules_broken += 1

    if datapoint["Accel_status"] != 1:
        rules_broken += 1

    if datapoint["Baro_status"] != 1:
        rules_broken += 1
    return rules_broken

<h3>ENGINE<h3>

In [5]:
engine_rule_alerts = []

for i in range (len(engine1_df)):
    alerts = 0

    alerts += (constant_rule_checker(engine1_df.iloc[i]))

    if engine1_df.iloc[i]["PHASE"] == "INITIALISATION":
        alerts += (initialisation_rule_checker(engine1_df.iloc[i]))
    elif engine1_df.iloc[i]["PHASE"] == "TAKEOFF":
        alerts += (takeoff_rule_checker(engine1_df.iloc[i]))
    elif engine1_df.iloc[i]["PHASE"] == "ON MISSION":
        alerts += (on_mission_rule_checker(engine1_df.iloc[i]))
    elif engine1_df.iloc[i]["PHASE"] == "RETURN TO ORIGIN":
        alerts += (return_to_origin_rule_checker(engine1_df.iloc[i]))
    elif engine1_df.iloc[i]["PHASE"] == "LANDING":
        alerts += (landing_rule_checker(engine1_df.iloc[i]))

    engine_rule_alerts.append(alerts)
    
count = 0
for value in engine_rule_alerts:
    if value >0:
        count +=1


#predict for engine

engine_kmeans = []
engine_lof = []
engine_svm = []
engine_db = []
engine_optics = []

base_df2 = base_df.copy()
base_df2['cluster'] = kmeans.labels_
engine1_df2 = clustering_engine1_df.copy()
engine1_df2['cluster'] = kmeans.predict(clustering_engine1_df)

centroids = kmeans.cluster_centers_

original_distances = []
for i in range(len(base_df)):
    centroid = centroids[int(base_df2.iloc[i]['cluster'])]
    distance = np.sqrt(np.sum((base_df.iloc[i] - centroid)**2))
    original_distances.append(distance)

original_mean = sum(original_distances)/len(original_distances)

original_std = np.std(original_distances)

threshold = original_mean + 3* original_std

for i in range(len(clustering_engine1_df)):
    centroid = centroids[int(engine1_df2.iloc[i]['cluster'])]
    distance = np.sqrt(np.sum((clustering_engine1_df.iloc[i] - centroid)**2))
    if distance > threshold:
        engine_kmeans.append(-1)
    else:
        engine_kmeans.append(1)


for i in range(clustering_engine1_df.shape[0]):
    prediction = lof.predict(clustering_engine1_df.iloc[[i]])
    for value in prediction:
        engine_lof.append(value)

for i in range(clustering_engine1_df.shape[0]):
    prediction = model.predict(clustering_engine1_df.iloc[[i]])
    for value in prediction:
        engine_svm.append(value)

for value in db.fit_predict(clustering_engine1_df):
    engine_db.append(value)

for value in optics_clustering.fit_predict(clustering_engine1_df):
    engine_optics.append(value)


#get predictions for engine using voting

final_engine_predictions = []

for i in range (len(engine1_df)):
    predictions = [engine_kmeans[i], engine_lof[i], engine_svm[i], engine_db[i], engine_optics[i]]
    final_engine_predictions.append(max(set(predictions), key=predictions.count))
    
overall_engine_count = 0

for i in range(len(engine1_df)):
    if final_engine_predictions[i] <0 or engine_rule_alerts[i] > 0:
        overall_engine_count+=1

print("For engine, out of the", len(engine1_df), "anomalies", overall_engine_count ,"were detected, which is", overall_engine_count/len(engine1_df)*100,"%")


/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warni

For engine, out of the 470 anomalies 320 were detected, which is 68.08510638297872 %
